In [2]:
## import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc4
import tarfile
import json
import uuid as uuid
import os
from datetime import datetime, timedelta
from pathlib import Path

## import ECCO utils
import sys
sys.path.append('/Users/mzahn/github_others/ECCOv4-py')
import ecco_v4_py as ecco

In [84]:
def iter_from_timestamp(timestamp_str):
    """
    takes the model timestamp and generates the model iteration number
    """
    
    ## Start time of the model is 5790000 (22.0319 years after 1992-01-01)
    ## there are 120 seconds for each iteration and 86400 seconds per day
    ## take the iteration number, convert to seconds, and calculate number of days since start of model
    
    date = np.datetime64(timestamp_str)
    
    model_start_time = np.datetime64("1992-01-01") # data.cal start time is 1992-01-01
    timestamp = date - model_start_time
    
    iter_num = int(np.array(timestamp, dtype='int')*86400/120)

    return iter_num

In [122]:
def timestamp_from_iter_num(iter_num):
    """
    takes the model iteration that was pulled from the data's filename and converts it to its equivalent datetime
    """
    ## Start time of the model is 5790000 (22.0319 years after 1992-01-01)
    ## there are 120 seconds for each iteration and 86400 seconds per day
    ## take the iteration number, convert to seconds, and calculate number of days since start of model
    
    
    ## from Mike: "Near the end of the simulation, I ran into some sort of instability so I changed the time step from 120 seconds to 60 seconds.
    ## Usually I would change it back to 120 second after getting past the instability but I was kinda close to the end so I just let it ride with 60 seconds."
    if iter_num > 1e7:
        iter_num = iter_num/2
    
    num_days_since_start = iter_num*120 / 86400 ## divide iter_number by 86400 which is equal to the number of seconds in a day
    
    model_start_time = datetime(1992,1,1) # data.cal start time is 1992-01-01
    timestamp = np.array([model_start_time + timedelta(days=num_days_since_start)], dtype='datetime64[ns]')
    
    return timestamp

In [86]:
iter_from_timestamp("2014-04-24")

5867280

Look into that date's gz file

In [126]:
data_dir = Path("/Users/mzahn/data/SASSIE/SASSIE_examples/daily_avg/ocean_state_3D_day_mean/ocean_state_3D_day_mean.0005860000/")

data_files = np.sort(list(data_dir.glob('*.data')))

In [131]:
timestamp_from_iter_num(5869440)

array(['2014-04-27T00:00:00.000000000'], dtype='datetime64[ns]')

In [128]:
for file in data_files:
    iter_num = int(str(file).split(".")[-2])
    timestamp = timestamp_from_iter_num(iter_num)
    print(timestamp)

['2014-04-14T00:00:00.000000000']
['2014-04-15T00:00:00.000000000']
['2014-04-16T00:00:00.000000000']
['2014-04-17T00:00:00.000000000']
['2014-04-18T00:00:00.000000000']
['2014-04-19T00:00:00.000000000']
['2014-04-20T00:00:00.000000000']
['2014-04-21T00:00:00.000000000']
['2014-04-22T00:00:00.000000000']
['2014-04-23T00:00:00.000000000']
['2014-04-24T00:00:00.000000000']
['2014-04-26T00:00:00.000000000']
['2014-04-27T00:00:00.000000000']


These dates are one day ahead because the iter number indicates the end of the averaging period and what is saved in the netCDF is the middle of the averaging period

In [ ]:
timestamp_from_iter_num(iter_num)